## load libraries

In [580]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import numpy.ma as ma
from scipy import stats
import scipy.spatial.distance as sp_distance
from sklearn.svm import NuSVC
import nibabel as nib
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.masking import intersect_masks
from sklearn import preprocessing
import seaborn as sns 
from brainiak.isfc import isc
from brainiak.isfc import isfc
from brainiak.fcma.util import compute_correlation
import brainiak.funcalign.srm
#from brainiak import image, io
import pandas as pd
import matplotlib.pyplot as plt
from pandas import *
from nilearn.masking import apply_mask
from nilearn.image import resample_to_img
from nipype.interfaces import afni
from nipy.labs.mask import intersect_masks
from scipy.stats import zscore, norm
from sklearn import linear_model
#from isc_reorder import isc_reorder
#from isc_recall import isc_recall
import pickle
from nilearn import image
import igraph
import bct
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.formula.api as smf

import multiprocessing as mp
import time

from nilearn.plotting import plot_roi, show
import patsy

%autosave 5
%matplotlib inline

Autosaving every 5 seconds


## set variables

In [3189]:
data_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/cartoon_recall_fmriprep_2020/fmriprep/'
analysis_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/'
roi_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/cartoon_recall_fmriprep_2020/freesurfer/HCP-MMP1_subject_atlas/'
roi_list = pd.read_excel(analysis_dir+'roilist.xlsx')
movie_list = ['barmaid','bluestreets']
subj_recall=pd.read_excel(analysis_dir+'subj_recall_sessions_run.xlsx')
event_boundaries = pd.read_excel(analysis_dir+'fmri_recall_event_boundaries_14events_each.xlsx')
subjlist_full = pd.read_excel(analysis_dir+'subjlist.xlsx')
hc_list = ['_L_ant_hippo.nii.gz','_L_post_hippo.nii.gz','_R_ant_hippo.nii.gz','_R_post_hippo.nii.gz']
## import node names
glasser_nodes=pd.read_excel('hc_glasser_nodes.xlsx')
recall_scoring = pd.read_excel(analysis_dir+'within_subj_fc/event detail scoring 14 events.xlsx')

num_subs = 24

In [3190]:
# load in univariate activity

boundary_diff_hc = np.load('/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/stats/boundary_diff.npy')

In [3191]:
# the order of the ROIs from Barnett et al (2021) PLoS Biol
indsort_sub = np.loadtxt(analysis_dir+'indsort_subnetworks.txt')
indsort_sub=indsort_sub.astype('int32')
for i in range(len(indsort_sub)):
    indsort_sub[i] = indsort_sub[i] -1

indsort_sub =np.append(indsort_sub,[358,359,360,361])

In [3192]:
## make array of subject ids
subject_list = subjlist_full['subj_id'].unique()
## set net labels
net_label = ['VIS','SOM-MOT','SAL1','DAN1','SAL2','DAN2','LANG','FPN','AUD','ATN','MTN','PMN','MPN','antHC','postHC']

In [3233]:
# event boundaries: first barmaid, then bluestreets
barmaid_boundaries = np.array([[0,     58,    97,    160,   189,   300, 350, 470, 507, 552, 619, 695, 772, 871, 1006]])
barmaid_boundaries = ((barmaid_boundaries)/1.22)+5 # adjust for TR and HRF

bluestreets_boundaries = np.array([[0,    75,   150,   226,   307, 381, 444, 526, 578, 665, 756, 828, 853,873,982]])
bluestreets_boundaries = ((bluestreets_boundaries)/1.22)+5 # adjust for TR and HRF

## load in fc data

In [3194]:
subject_list

array(['sub-000', 'sub-101', 'sub-102', 'sub-103', 'sub-104', 'sub-105',
       'sub-106', 'sub-108', 'sub-109', 'sub-111', 'sub-112', 'sub-115',
       'sub-117', 'sub-118', 'sub-119', 'sub-120', 'sub-121', 'sub-123',
       'sub-124', 'sub-126', 'sub-127', 'sub-128', 'sub-129', 'sub-130'],
      dtype=object)

In [3195]:
subj_barmaid_movie_fc = np.load(analysis_dir+'subj_barmaid_movie_fc_full.npy')
subj_bluestreets_movie_fc = np.load(analysis_dir+'subj_bluestreets_movie_fc_full.npy')
# fisher z-transform

subj_barmaid_movie_fc_z= np.arctanh(subj_barmaid_movie_fc)
subj_bluestreets_movie_fc_z = np.arctanh(subj_bluestreets_movie_fc)

for x in range(362):
    subj_barmaid_movie_fc_z[:,x,x] = np.nan
    subj_bluestreets_movie_fc_z[:,x,x] = np.nan

In [3196]:
np.shape(subj_bluestreets_movie_fc_z)

(24, 362, 362)

In [3198]:
barmaid_movie_fc_data = np.load(analysis_dir+'barmaid_movie_data_extracted_full.npy')
bluestreets_movie_fc_data = np.load(analysis_dir+'bluestreets_movie_data_extracted_full.npy')

In [3199]:
num_subs = 24

## calc fc at boundaries vs. non

In [1669]:
net_label_sep_hc = ['VIS',
 'SOM-MOT',
 'SAL1',
 'DAN1',
 'SAL2',
 'DAN2',
 'LANG',
 'FPN',
 'AUD',
 'ATN',
 'MTN',
 'PMN',
 'MPN',
 'LantHC',
 'LpostHC',
 'RantHC',
 'RpostHC']

In [1686]:
temp= glasser_nodes['community_sublabels'].values

In [1690]:
temp[360] = 16
temp[361] = 17

In [1691]:
glasser_nodes['community_sublabels_sep_hc']=temp

## post boundary

In [3234]:
event_list = range(0,14)

subject = []
events_recalled = []
central_d = []
peri_d = []
movie = []
event_length = []
delay = []
mov = 0 ## barmaid
events=[]
fc = []
net1_label=[]
net2_label = []
event_label = []
window_size = 10

for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int(np.round(barmaid_boundaries[0,event+1]))-window_size
        end = int(np.round(barmaid_boundaries[0,event+1]))+window_size
        event_fc_data = barmaid_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(np.size(net_label)):
            for net2 in range(np.size(net_label)):
                net1_label.append(net_label[net1])
                net2_label.append(net_label[net2])
                net_to_net_fc = event_fc_z[glasser_nodes['community_sublabels']==net1+1][:,glasser_nodes['community_sublabels']==net2+1]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(int(np.round(barmaid_boundaries[0,event+1]))-int(np.round(barmaid_boundaries[0,event])))
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)

event_list = range(0,14)
mov=1  # run bluestreets 
for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int(np.round(bluestreets_boundaries[0,event+1]))-window_size
        end = int(np.round(bluestreets_boundaries[0,event+1]))+window_size
        event_fc_data = bluestreets_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(np.size(net_label)):
            for net2 in range(np.size(net_label)):
                net1_label.append(net_label[net1])
                net2_label.append(net_label[net2])
                net_to_net_fc = event_fc_z[glasser_nodes['community_sublabels']==net1+1][:,glasser_nodes['community_sublabels']==net2+1]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(int(np.round(bluestreets_boundaries[0,event+1]))-int(np.round(bluestreets_boundaries[0,event])))
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)       

In [3235]:
delay = np.reshape(delay,np.shape(delay)[0])
percent_events = np.reshape(percent_events,np.shape(percent_events)[0])
event_recalled = np.reshape(event_recalled,np.shape(event_recalled)[0])
peri_d = np.reshape(peri_d,np.shape(peri_d)[0])
central_d = np.reshape(central_d,np.shape(central_d)[0])

data3= np.transpose([subject,movie,events,delay,fc,net1_label,net2_label,event_label,event_length,event_recalled,peri_d,central_d])
columns3 = ['subject','movie','events','delay','fc','net1_label','net2_label','event_label','event_length','event_recalled','peri_d','central_d']

eventboundary_encoding_fc_recall_pd = pandas.DataFrame(data =data3, columns = columns3)

eventboundary_encoding_fc_recall_pd['fc'] = eventboundary_encoding_fc_recall_pd['fc'].astype('float')
eventboundary_encoding_fc_recall_pd['event_length'] = eventboundary_encoding_fc_recall_pd['event_length'].astype('float')
eventboundary_encoding_fc_recall_pd['peri_d'] = eventboundary_encoding_fc_recall_pd['peri_d'].astype('float')
eventboundary_encoding_fc_recall_pd['central_d'] = eventboundary_encoding_fc_recall_pd['central_d'].astype('float')
eventboundary_encoding_fc_recall_pd['event_recalled'] = eventboundary_encoding_fc_recall_pd['event_recalled'].astype('float')
eventboundary_encoding_fc_recall_pd['delay'] = eventboundary_encoding_fc_recall_pd['delay'].astype('str')

In [ ]:
#offset
eventboundary_encoding_fc_recall_pd = pd.read_csv(analysis_dir+'/within_subj_fc/eventboundary_encoding_fc_recall_pd.csv')

## pre event boundary

In [3156]:
event_list = range(0,14)
num_subs = 24

subject = []
events_recalled = []
central_d = []
peri_d = []
movie = []
event_length = []
delay = []
mov = 0 ## barmaid
events=[]
fc = []
net1_label=[]
net2_label = []
event_label = []
event_recalled =[]
peri_d=[]
central_d=[]
window_size = 10

for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int(np.round(barmaid_boundaries[0,event]))-window_size
        if start<0:
             start=0
        else:
            start=start
        end = int(np.round(barmaid_boundaries[0,event]))+window_size
        event_fc_data = barmaid_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(np.size(net_label)):
            for net2 in range(np.size(net_label)):
                net1_label.append(net_label[net1])
                net2_label.append(net_label[net2])
                net_to_net_fc = event_fc_z[glasser_nodes['community_sublabels']==net1+1][:,glasser_nodes['community_sublabels']==net2+1]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(int(np.round(barmaid_boundaries[0,event+1]))-int(np.round(barmaid_boundaries[0,event])))
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)

event_list = range(0,14)
mov=1  # run bluestreets 
for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int(np.round(bluestreets_boundaries[0,event]))-window_size
        if start<0:
             start=0
        else:
            start=start
        end = int(np.round(bluestreets_boundaries[0,event]))+window_size
        event_fc_data = bluestreets_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(np.size(net_label)):
            for net2 in range(np.size(net_label)):
                net1_label.append(net_label[net1])
                net2_label.append(net_label[net2])
                net_to_net_fc = event_fc_z[glasser_nodes['community_sublabels']==net1+1][:,glasser_nodes['community_sublabels']==net2+1]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+14+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(int(np.round(bluestreets_boundaries[0,event+1]))-int(np.round(bluestreets_boundaries[0,event])))
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values) 

In [3157]:
pre_eventboundary_encoding_fc_recall_pd['central_d'] = eventboundary_encoding_fc_recall_pd['central_d'].astype('float')
pre_eventboundary_encoding_fc_recall_pd['peri_d'] = eventboundary_encoding_fc_recall_pd['peri_d'].astype('float')

In [3158]:
delay = np.reshape(delay,np.shape(delay)[0])
percent_events = np.reshape(percent_events,np.shape(percent_events)[0])
event_recalled = np.reshape(event_recalled,np.shape(event_recalled)[0])
central_d = np.reshape(central_d,np.shape(central_d)[0])
peri_d = np.reshape(peri_d,np.shape(peri_d)[0])
data3= np.transpose([subject,movie,events,delay,fc,net1_label,net2_label,event_label,event_length,event_recalled,central_d,peri_d,path_length])
columns3 = ['subject','movie','events','delay','fc','net1_label','net2_label','event_label','event_length','event_recalled','central_d','peri_d','path_length']

pre_eventboundary_encoding_fc_recall_pd = pandas.DataFrame(data =data3, columns = columns3)


pre_eventboundary_encoding_fc_recall_pd['fc'] = pre_eventboundary_encoding_fc_recall_pd['fc'].astype('float')
pre_eventboundary_encoding_fc_recall_pd['central_d'] = eventboundary_encoding_fc_recall_pd['central_d'].astype('float')
pre_eventboundary_encoding_fc_recall_pd['peri_d'] = eventboundary_encoding_fc_recall_pd['peri_d'].astype('float')
pre_eventboundary_encoding_fc_recall_pd['path_length'] = eventboundary_encoding_fc_recall_pd['path_length'].astype('float')
pre_eventboundary_encoding_fc_recall_pd['event_recalled'] = pre_eventboundary_encoding_fc_recall_pd['event_recalled'].astype('float')
pre_eventboundary_encoding_fc_recall_pd['event_length'] = eventboundary_encoding_fc_recall_pd['event_length'].astype('float')

In [530]:
pre_eventboundary_encoding_fc_recall_pd.to_csv(analysis_dir+'pre_eventboundary_encoding_fc_recall_pd.csv')

In [2791]:
#onset
pre_eventboundary_encoding_fc_recall_pd = pd.read_csv(analysis_dir+'pre_eventboundary_encoding_fc_recall_pd.csv')

## event middle

In [3243]:
# take in the arithmetic middle
event_list = range(0,14)

subject = []
events_recalled = []
central_d = []
peri_d = []
movie = []
event_length = []
delay = []
mov = 0 ## barmaid
events=[]
fc = []
net1_label=[]
net2_label = []
event_label = []
event_recalled=[]
window_size = 10

for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int((np.round(barmaid_boundaries[0,event])+np.round(barmaid_boundaries[0,event+1]))/2)-window_size
        end = int((np.round(barmaid_boundaries[0,event])+np.round(barmaid_boundaries[0,event+1]))/2)+window_size
        event_fc_data = barmaid_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(np.size(net_label)):
            for net2 in range(np.size(net_label)):
                net1_label.append(net_label[net1])
                net2_label.append(net_label[net2])
                net_to_net_fc = event_fc_z[glasser_nodes['community_sublabels']==net1+1][:,glasser_nodes['community_sublabels']==net2+1]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(end-start)
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)


event_list = range(0,14)
mov=1  # run bluestreets 
for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int((np.round(bluestreets_boundaries[0,event])+np.round(bluestreets_boundaries[0,event+1]))/2)-window_size
        end = int((np.round(bluestreets_boundaries[0,event])+np.round(bluestreets_boundaries[0,event+1]))/2)+window_size
        event_fc_data = bluestreets_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(np.size(net_label)):
            for net2 in range(np.size(net_label)):
                net1_label.append(net_label[net1])
                net2_label.append(net_label[net2])
                net_to_net_fc = event_fc_z[glasser_nodes['community_sublabels']==net1+1][:,glasser_nodes['community_sublabels']==net2+1]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(end-start)
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)

In [3244]:
delay = np.reshape(delay,np.shape(delay)[0])
percent_events = np.reshape(percent_events,np.shape(percent_events)[0])
event_recalled = np.reshape(event_recalled,np.shape(percent_events)[0])
peri_d = np.reshape(peri_d,np.shape(peri_d)[0])
central_d = np.reshape(central_d,np.shape(central_d)[0])

In [3245]:
data4= np.transpose([subject,movie,events,delay,fc,net1_label,net2_label,event_label,event_length,event_recalled,central_d,peri_d,path_length])
columns4 = ['subject','movie','events','delay','fc','net1_label','net2_label','event_label','event_length','event_recalled','central_d','peri_d','path_length']

In [3246]:
nonboundary_encoding_fc_recall_pd['central_d'] = eventboundary_encoding_fc_recall_pd['central_d'].astype('float')
nonboundary_encoding_fc_recall_pd['peri_d'] = eventboundary_encoding_fc_recall_pd['peri_d'].astype('float')

In [3247]:
nonboundary_encoding_fc_recall_pd = pandas.DataFrame(data =data4, columns = columns4)

nonboundary_encoding_fc_recall_pd['delay'] = nonboundary_encoding_fc_recall_pd['delay'].astype('int')
nonboundary_encoding_fc_recall_pd['fc'] = nonboundary_encoding_fc_recall_pd['fc'].astype('float')
nonboundary_encoding_fc_recall_pd['event_recalled'] = nonboundary_encoding_fc_recall_pd['event_recalled'].astype('float')
nonboundary_encoding_fc_recall_pd['central_d'] = eventboundary_encoding_fc_recall_pd['central_d'].astype('float')
nonboundary_encoding_fc_recall_pd['peri_d'] = eventboundary_encoding_fc_recall_pd['peri_d'].astype('float')
nonboundary_encoding_fc_recall_pd['event_length'] = eventboundary_encoding_fc_recall_pd['event_length'].astype('float')

In [2438]:
#middle
nonboundary_encoding_fc_recall_pd.to_csv(analysis_dir+'within_subj_fc/event_middle_encoding_fc_recall_pd.csv')

# combine event_start event_middle and offset into 1 dataframe

In [4328]:
# set labels for net2_label to investigate hc-network FC
model_data1 =eventboundary_encoding_fc_recall_pd[eventboundary_encoding_fc_recall_pd['net1_label']=='postHC'][eventboundary_encoding_fc_recall_pd['net2_label']=='PMN']
model_data2 = eventboundary_encoding_fc_recall_pd[eventboundary_encoding_fc_recall_pd['net1_label']=='antHC'][eventboundary_encoding_fc_recall_pd['net2_label']=='PMN']


model_data3 = pd.concat([model_data1, model_data2])

model_data4 =pre_eventboundary_encoding_fc_recall_pd[pre_eventboundary_encoding_fc_recall_pd['net1_label']=='postHC'][pre_eventboundary_encoding_fc_recall_pd['net2_label']=='PMN']
model_data5 = pre_eventboundary_encoding_fc_recall_pd[pre_eventboundary_encoding_fc_recall_pd['net1_label']=='antHC'][pre_eventboundary_encoding_fc_recall_pd['net2_label']=='PMN']

model_data6 = pd.concat([model_data4, model_data5])

model_data7 =nonboundary_encoding_fc_recall_pd[nonboundary_encoding_fc_recall_pd['net1_label']=='antHC'][nonboundary_encoding_fc_recall_pd['net2_label']=='PMN']
model_data8 =nonboundary_encoding_fc_recall_pd[nonboundary_encoding_fc_recall_pd['net1_label']=='postHC'][nonboundary_encoding_fc_recall_pd['net2_label']=='PMN']

model_data9 = pd.concat([model_data7, model_data8])

model_data3['window']='offset'
model_data6['window']='event_start'
model_data9['window']='event_middle'

boundary_and_nonboundary_encoding_fc_pd = pd.concat([model_data3,model_data6,model_data9], axis = 0,ignore_index=True)

In [4329]:
## mean center FC within participant

model_data_sub_mean =boundary_and_nonboundary_encoding_fc_pd.groupby('subject').mean()
   
fc_mc = []
for i in range(np.shape(boundary_and_nonboundary_encoding_fc_pd)[0]):
    fc_mc.append(boundary_and_nonboundary_encoding_fc_pd.iloc[i]['fc'] - model_data_sub_mean.loc[boundary_and_nonboundary_encoding_fc_pd.iloc[i]['subject']]['fc'])

    

    
    
boundary_and_nonboundary_encoding_fc_pd['fc_mc'] =fc_mc
boundary_and_nonboundary_encoding_fc_pd['delay'] = boundary_and_nonboundary_encoding_fc_pd['delay'].astype('str')
boundary_and_nonboundary_encoding_fc_pd['delay'] = boundary_and_nonboundary_encoding_fc_pd['delay'].astype('category')

In [4330]:
boundary_and_nonboundary_encoding_fc_pd['boundary_diff'] = np.concatenate([boundary_diff_hc,boundary_diff_hc,boundary_diff_hc])

In [4331]:
boundary_and_nonboundary_encoding_fc_pd.to_csv(analysis_dir+'within_subj_fc/boundary_and_nonboundary_encoding_fc_pd_PMN.csv')

## ROI level boundary ~ memory effects

In [3788]:
# offset
event_list = range(0,14)

subject = []
events_recalled = []
central_d = []
peri_d = []
movie = []
event_length = []
delay = []
mov = 0 ## barmaid
events=[]
fc = []
roi=[]
net2_label = []
event_label = []
event_recalled =[]
window_size = 10

for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int(np.round(barmaid_boundaries[0,event+1]))-window_size
        end = int(np.round(barmaid_boundaries[0,event+1]))+window_size
        event_fc_data = barmaid_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(358):
            for net2 in range(14,16):
                roi.append(net1)
                net2_label.append(net_label[net2-1])
                net_to_net_fc = event_fc_z[net1,glasser_nodes['community_sublabels']==net2]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(int(np.round(barmaid_boundaries[0,event+1]))-int(np.round(barmaid_boundaries[0,event])))
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)

event_list = range(0,14)
mov=1  # run bluestreets 
for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int(np.round(bluestreets_boundaries[0,event+1]))-window_size
        end = int(np.round(bluestreets_boundaries[0,event+1]))+window_size
        event_fc_data = bluestreets_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(358):
            for net2 in range(14,16):
                roi.append(net1)
                net2_label.append(net_label[net2-1])
                net_to_net_fc = event_fc_z[net1,glasser_nodes['community_sublabels']==net2]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(int(np.round(bluestreets_boundaries[0,event+1]))-int(np.round(bluestreets_boundaries[0,event])))
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)
        
        

In [3791]:
# set up dataframe
delay = np.reshape(delay,np.shape(delay)[0])
percent_events = np.reshape(percent_events,np.shape(percent_events)[0])
event_recalled = np.reshape(event_recalled,np.shape(event_recalled)[0])
peri_d = np.reshape(peri_d,np.shape(peri_d)[0])
central_d = np.reshape(central_d,np.shape(central_d)[0])

data3= np.transpose([subject,movie,events,delay,fc,roi,net2_label,event_label,event_length,event_recalled,peri_d,central_d])
columns3 = ['subject','movie','events','delay','fc','roi','net2_label','event_label','event_length','event_recalled','peri_d','central_d']

roi_eventboundary_encoding_fc_recall_pd = pandas.DataFrame(data =data3, columns = columns3)


roi_eventboundary_encoding_fc_recall_pd['fc'] = roi_eventboundary_encoding_fc_recall_pd['fc'].astype('float')
roi_eventboundary_encoding_fc_recall_pd['event_length'] = roi_eventboundary_encoding_fc_recall_pd['event_length'].astype('float')
roi_eventboundary_encoding_fc_recall_pd['peri_d'] = roi_eventboundary_encoding_fc_recall_pd['peri_d'].astype('float')
roi_eventboundary_encoding_fc_recall_pd['central_d'] = roi_eventboundary_encoding_fc_recall_pd['central_d'].astype('float')
roi_eventboundary_encoding_fc_recall_pd['event_recalled'] = roi_eventboundary_encoding_fc_recall_pd['event_recalled'].astype('float')

roi_eventboundary_encoding_fc_recall_pd['total_details']=roi_eventboundary_encoding_fc_recall_pd['peri_d']+roi_eventboundary_encoding_fc_recall_pd['central_d']
roi_eventboundary_encoding_fc_recall_pd['total_details_log'] = np.log(roi_eventboundary_encoding_fc_recall_pd['total_details'])

 
model_data_sub_mean =roi_eventboundary_encoding_fc_recall_pd.groupby('subject').mean()
   
    
fc_mc = []
for i in range(np.shape(roi_eventboundary_encoding_fc_recall_pd)[0]):
    fc_mc.append(roi_eventboundary_encoding_fc_recall_pd.iloc[i]['fc'] - model_data_sub_mean.loc[roi_eventboundary_encoding_fc_recall_pd.iloc[i]['subject']]['fc'])
    
roi_eventboundary_encoding_fc_recall_pd['fc_mc'] =fc_mc

In [ ]:
#save dataframe
roi_eventboundary_encoding_fc_recall_pd.to_csv(analysis_dir+'within_subj_fc/roi_eventboundary_encoding_fc_recall_pd.csv')

In [3793]:
# middle
event_list = range(0,14)

subject = []
events_recalled = []
central_d = []
peri_d = []
movie = []
event_length = []
delay = []
mov = 0 ## barmaid
events=[]
fc = []
roi=[]
net2_label = []
event_label = []
event_recalled =[]
window_size = 10

for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int((np.round(barmaid_boundaries[0,event])+np.round(barmaid_boundaries[0,event+1]))/2)-window_size
        end = int((np.round(barmaid_boundaries[0,event])+np.round(barmaid_boundaries[0,event+1]))/2)+window_size
        event_fc_data = barmaid_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(358):
            for net2 in range(14,16):
                roi.append(net1)
                net2_label.append(net_label[net2-1])
                net_to_net_fc = event_fc_z[net1,glasser_nodes['community_sublabels']==net2]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(int(np.round(barmaid_boundaries[0,event+1]))-int(np.round(barmaid_boundaries[0,event])))
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)

event_list = range(0,14)
mov=1  # run bluestreets 
for sub in range(num_subs):
    temp1 = recall_scoring[recall_scoring['subject']==subject_list[sub]][recall_scoring['movie']==movie_list[mov]]
    for event in range(14):
        temp2 = temp1[temp1['event']==event+1]
        start = int((np.round(bluestreets_boundaries[0,event])+np.round(bluestreets_boundaries[0,event+1]))/2)-window_size
        end = int((np.round(bluestreets_boundaries[0,event])+np.round(bluestreets_boundaries[0,event+1]))/2)+window_size
        event_fc_data = bluestreets_movie_fc_data[sub,:,start:end]
        event_fc = np.corrcoef(event_fc_data)
        event_fc_z= np.arctanh(event_fc)
        for x in range(np.shape(event_fc_z)[0]):
            event_fc_z[x,x]=np.nan
        for net1 in range(358):
            for net2 in range(14,16):
                roi.append(net1)
                net2_label.append(net_label[net2-1])
                net_to_net_fc = event_fc_z[net1,glasser_nodes['community_sublabels']==net2]
                fc.append(np.nanmean(net_to_net_fc.flatten()))
                subject.append(subject_list[sub])
                events.append(event_list[event]+1)
                event_label.append(event_list[event]+1)
                delay.append(subjlist_full[subjlist_full['subj_id']==subject_list[sub]][movie_list[mov]+'_delay'].values)
                movie.append(movie_list[mov])
                event_length.append(int(np.round(bluestreets_boundaries[0,event+1]))-int(np.round(bluestreets_boundaries[0,event])))
                event_recalled.append(event_boundaries[event_boundaries['subject']==subject_list[sub]][event_boundaries['movie']==movie_list[mov]][event_boundaries['event']==event+1]['recalled'].values)
                peri_d.append(temp2['total pd'].values)
                central_d.append(temp2['total cd'].values)

In [3794]:
# set up dataframe
delay = np.reshape(delay,np.shape(delay)[0])
percent_events = np.reshape(percent_events,np.shape(percent_events)[0])
event_recalled = np.reshape(event_recalled,np.shape(event_recalled)[0])
peri_d = np.reshape(peri_d,np.shape(peri_d)[0])
central_d = np.reshape(central_d,np.shape(central_d)[0])

data3= np.transpose([subject,movie,events,delay,fc,roi,net2_label,event_label,event_length,event_recalled,peri_d,central_d])
columns3 = ['subject','movie','events','delay','fc','roi','net2_label','event_label','event_length','event_recalled','peri_d','central_d']

roi_eventboundary_encoding_fc_recall_middle_pd = pandas.DataFrame(data =data3, columns = columns3)


roi_eventboundary_encoding_fc_recall_middle_pd['fc'] = roi_eventboundary_encoding_fc_recall_middle_pd['fc'].astype('float')
roi_eventboundary_encoding_fc_recall_middle_pd['event_length'] = roi_eventboundary_encoding_fc_recall_middle_pd['event_length'].astype('float')
roi_eventboundary_encoding_fc_recall_middle_pd['peri_d'] = roi_eventboundary_encoding_fc_recall_middle_pd['peri_d'].astype('float')
roi_eventboundary_encoding_fc_recall_middle_pd['central_d'] = roi_eventboundary_encoding_fc_recall_middle_pd['central_d'].astype('float')
roi_eventboundary_encoding_fc_recall_middle_pd['event_recalled'] = roi_eventboundary_encoding_fc_recall_middle_pd['event_recalled'].astype('float')
roi_eventboundary_encoding_fc_recall_middle_pd['total_details']=roi_eventboundary_encoding_fc_recall_middle_pd['peri_d']+roi_eventboundary_encoding_fc_recall_middle_pd['central_d']
roi_eventboundary_encoding_fc_recall_middle_pd['total_details_log'] = np.log(roi_eventboundary_encoding_fc_recall_middle_pd['total_details'])

 
model_data_sub_mean =roi_eventboundary_encoding_fc_recall_middle_pd.groupby('subject').mean()
   
    
fc_mc = []
for i in range(np.shape(roi_eventboundary_encoding_fc_recall_middle_pd)[0]):
    fc_mc.append(roi_eventboundary_encoding_fc_recall_middle_pd.iloc[i]['fc'] - model_data_sub_mean.loc[roi_eventboundary_encoding_fc_recall_middle_pd.iloc[i]['subject']]['fc'])
roi_eventboundary_encoding_fc_recall_middle_pd['fc_mc'] =fc_mc

In [3795]:
#save dataframe
roi_eventboundary_encoding_fc_recall_middle_pd.to_csv(analysis_dir+'within_subj_fc/roi_eventboundary_encoding_fc_recall_middle_pd.csv')

# create dataframes for event details

In [1822]:
eventboundary_encoding_fc_recall_pd

,Unnamed: 0,subject,movie,events,delay,fc,net1_label,net2_label,event_label,event_length,percent_events,event_recalled,peri_d,central_d,path_length,true_pd,true_cd
0,0,sub-000,barmaid,1,0,0.291329,VIS,VIS,1,48.0,0.714286,1.0,6.0,4.0,1.995478,5.0,3.5
1,1,sub-000,barmaid,1,0,-0.125273,VIS,SOM-MOT,1,48.0,0.714286,1.0,6.0,4.0,2.571972,5.0,3.5
2,2,sub-000,barmaid,1,0,-0.145123,VIS,SAL1,1,48.0,0.714286,1.0,6.0,4.0,2.587646,5.0,3.5
3,3,sub-000,barmaid,1,0,0.445576,VIS,DAN1,1,48.0,0.714286,1.0,6.0,4.0,1.820428,5.0,3.5
4,4,sub-000,barmaid,1,0,-0.381864,VIS,SAL2,1,48.0,0.714286,1.0,6.0,4.0,2.888138,5.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151195,151195,sub-130,bluestreets,14,0,-0.173646,postHC,MTN,14,89.0,0.854286,1.0,5.5,3.5,2.367175,8.0,4.0
151196,151196,sub-130,bluestreets,14,0,-0.686524,postHC,PMN,14,89.0,0.854286,1.0,5.5,3.5,2.630863,8.0,4.0
151197,151197,sub-130,bluestreets,14,0,-0.137277,postHC,MPN,14,89.0,0.854286,1.0,5.5,3.5,2.330786,8.0,4.0
151198,151198,sub-130,bluestreets,14,0,0.965930,postHC,antHC,14,89.0,0.854286,1.0,5.5,3.5,1.628305,8.0,4.0


In [3648]:
model_data1 =eventboundary_encoding_fc_recall_pd[eventboundary_encoding_fc_recall_pd['net1_label']=='postHC'][eventboundary_encoding_fc_recall_pd['net2_label']=='PMN']
model_data2 = eventboundary_encoding_fc_recall_pd[eventboundary_encoding_fc_recall_pd['net1_label']=='antHC'][eventboundary_encoding_fc_recall_pd['net2_label']=='PMN']


model_data3 = pd.concat([model_data1, model_data2])

model_data4 =pre_eventboundary_encoding_fc_recall_pd[pre_eventboundary_encoding_fc_recall_pd['net1_label']=='postHC'][pre_eventboundary_encoding_fc_recall_pd['net2_label']=='PMN']
model_data5 = pre_eventboundary_encoding_fc_recall_pd[pre_eventboundary_encoding_fc_recall_pd['net1_label']=='antHC'][pre_eventboundary_encoding_fc_recall_pd['net2_label']=='PMN']

model_data6 = pd.concat([model_data4, model_data5])

model_data7 =nonboundary_encoding_fc_recall_pd[nonboundary_encoding_fc_recall_pd['net1_label']=='antHC'][nonboundary_encoding_fc_recall_pd['net2_label']=='PMN']
model_data8 =nonboundary_encoding_fc_recall_pd[nonboundary_encoding_fc_recall_pd['net1_label']=='postHC'][nonboundary_encoding_fc_recall_pd['net2_label']=='PMN']

model_data9 = pd.concat([model_data7, model_data8])

model_data3['window']='offset'
model_data6['window']='event_onset'
model_data9['window']='event_middle'

boundary_and_nonboundary_encoding_fc_details = pd.concat([model_data3,model_data6,model_data9], axis = 0,ignore_index=True)

In [3649]:
boundary_and_nonboundary_encoding_fc_details['boundary_diff'] = np.concatenate([boundary_diff_hc,boundary_diff_hc,boundary_diff_hc])
boundary_and_nonboundary_encoding_fc_details['delay'] = boundary_and_nonboundary_encoding_fc_details['delay'].astype('str')
boundary_and_nonboundary_encoding_fc_details['delay'] = boundary_and_nonboundary_encoding_fc_details['delay'].astype('category')

In [3650]:
boundary_and_nonboundary_encoding_fc_details.to_csv(analysis_dir+'boundary_and_nonboundary_encoding_fc_details.csv')